In [1]:
import astropy.constants as const
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

import math

In [2]:
E_nu_ob = 1*u.PeV
K_p_gamma = 0.2
K_pi_e = 1/8
K_nu = 0.05
E_p_gamma_thresh = 0.3*u.GeV
B_crit = 4.41e13*u.gauss
r_m = (const.m_e / const.m_p).to(u.dimensionless_unscaled)
cross_section_p_gamma = 5e-28*u.cm**2

In [3]:
class Beam:
    
    def __init__(self, doppler_factor, theta, z):
        self.gamma = self.lorentz_factor(doppler_factor, theta)
        self.doppler = doppler_factor
        self.theta = theta
        self.z = z
        self.cosmo = FlatLambdaCDM(H0=69*(u.km/u.s/u.Mpc), Tcmb0=2.725*u.K, Om0=0.286)
        
    @staticmethod
    def lorentz_factor(doppler_factor, theta):
        theta = math.radians(theta)
        numerator = 1 - math.cos(theta)*math.sqrt(1 - (doppler_factor*math.sin(theta))**2)
        denominator = doppler_factor * math.sin(theta)**2
        return numerator / denominator
        
    def duration_to_comoving(self, duration):
        return duration * self.doppler / (1 + self.z)
    
    def energy_to_comoving(self, energy):
        return energy * (1 + self.z) / self.gamma
    
    def energy_to_observer(self, energy):
        return energy * self.gamma / (1 + self.z)
    
    def luminosity_distance(self):
        return self.cosmo.luminosity_distance(z)

In [4]:
beam_pks = Beam(40, 1, 0.432)
B_pks = 3e-2*u.gauss
R_pks = 5.5e16*u.cm
t_var_pks = 9*u.day

beam_ton = Beam(53, 1, 0.725)
B_ton = 3e-2*u.gauss
R_ton = 6e16*u.cm
t_var_ton = 2*u.day

In [5]:
# Calculate Hillas criterion

def hillas_criterion(beam, B, t_var=None, R=None):
    Z = 1
    beta = 1
    if R is None and t_var is not None:
        t_var_R = (const.c * beam.duration_to_comoving(t_var)).to(u.cm)
        print("    t_var = {} (R = {:.02})".format(t_var, t_var_R))
        L = const.c * beam.duration_to_comoving(t_var)
    elif t_var is None and R is not None:
        print("    R = {}".format(R))
        L = R
    else:
        raise ValueError("Exactly one of R and t_var must be specified")
    E_max = Z * const.e.si * beta * const.c * B * L
    print("Rest frame proton energy: {:.0f}".format(E_max.to(u.PeV)))
    E_nu = 0.05*beam.energy_to_observer(E_max)
    print("Observer frame neutrino energy: {:.0f}".format(E_nu.to(u.PeV)))


print("PKS 1222+216")
hillas_criterion(beam_pks, B_pks, R=R_pks)
hillas_criterion(beam_pks, B_pks, t_var=t_var_pks)
print("\nTon 599")
hillas_criterion(beam_ton, B_ton, R=R_ton)
hillas_criterion(beam_ton, B_ton, t_var=t_var_ton)

PKS 1222+216
    R = 5.5e+16 cm
Rest frame proton energy: 495 PeV
Observer frame neutrino energy: 403 PeV
    t_var = 9.0 d (R = 6.5e+17 cm)
Rest frame proton energy: 5856 PeV
Observer frame neutrino energy: 4768 PeV

Ton 599
    R = 6e+16 cm
Rest frame proton energy: 540 PeV
Observer frame neutrino energy: 601 PeV
    t_var = 2.0 d (R = 1.6e+17 cm)
Rest frame proton energy: 1432 PeV
Observer frame neutrino energy: 1594 PeV


In [6]:
# Calculate synchrotron peak energy
constants = const.c*const.e.si*(K_p_gamma * K_pi_e)**2 / (2*math.pi) / (const.m_e*const.c**2)**3
variables = (1.5)**-1 * 23 * 3e-2*u.gauss * (2*u.PeV)**2
(constants*variables).to(u.Hz, equivalencies=u.spectral())

<Quantity 1.2328191e+22 Hz>

In [7]:
# Calculate proton luminosity parameterized by the Eddington luminosity
constants = 0.1*16/27/K_p_gamma/K_pi_e/const.G*const.c**2*const.sigma_T/cross_section_p_gamma*E_p_gamma_thresh
variables = 23**2 * 6e16*u.cm / (5e10*const.M_sun) / (2*u.PeV) * 0.5
print("L = {:.03} L_Edd".format((constants * variables).to(u.dimensionless_unscaled)))

L = 1.02 L_Edd


In [8]:
# Calculate the expected number of neutrinos
def neutrino_count(nu_fnu_peak, duration):
    # Keivani et al. (2018), Eq.4
    effective_area = 1e6*u.cm**2
    
    count = 2 * nu_fnu_peak / E_nu_ob * math.log(10) * duration * effective_area
    return count.to(u.dimensionless_unscaled)

neutrino_count(1e-12*u.erg*u.cm**-2*u.s**-1, 5*u.day)

<Quantity 0.00124171>